In [6]:
from nms_cupy import cupy_call_nms_kernel,_load_kernel,_nms_gpu_code 
import numpy as np
import cupy as cp
from nms_numba_cpu import nms_cpu
from nms_numba import nms_kernel,numba_call_nms_kernel
from numba import cuda
thresh=0.7

1.ubuntu 18    
2.python 3.7    
3.cuda   9.2    
4.gpu  2080ti    

# 1.numba

In [8]:
bbox=np.load("bbox.npy")
n_bbox = bbox.shape[0]
threads_per_block = 64
col_blocks = np.ceil(n_bbox / threads_per_block).astype(np.int32)
blocks = (col_blocks, col_blocks, 1)
threads = (threads_per_block, 1, 1)

mask_dev = cuda.device_array((n_bbox * col_blocks,), dtype=np.uint64)
bbox=bbox.reshape(-1)
bbox =cuda.to_device(bbox)

%timeit nms_kernel[blocks, threads](n_bbox, bbox, mask_dev)


The slowest run took 225.82 times longer than the fastest. This could mean that an intermediate result is being cached.
24.4 ms ± 21.7 ms per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [26]:
bbox=np.load("bbox.npy")
%timeit numba_call_nms_kernel(bbox,0.7)

63.7 ms ± 458 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


# 2.cupy #

In [9]:
bbox=np.load("bbox.npy")
bbox=cp.asarray(bbox)
threads_per_block = 64
col_blocks = np.ceil(n_bbox / threads_per_block).astype(np.int32)
blocks = (col_blocks, col_blocks, 1)
threads = (threads_per_block, 1, 1)

mask_dev = cp.zeros((n_bbox * col_blocks,), dtype=np.uint64)
bbox = cp.ascontiguousarray(bbox, dtype=np.float32)
kern = _load_kernel('nms_kernel', _nms_gpu_code)
%timeit kern(blocks, threads, args=(cp.int32(n_bbox), cp.float32(thresh),bbox, mask_dev))

5.73 ms ± 19.8 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [22]:
bbox=np.load("bbox.npy")
bbox=cp.asarray(bbox)
%timeit cupy_call_nms_kernel(bbox,0.7)
# print(n_selec.get())

18.5 ms ± 435 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


# 3.numba_cpu #

In [27]:
bbox=np.load("bbox.npy")
%timeit keep=nms_cpu(bbox,0.7)
# print(keep.shape)

61.9 ms ± 1.62 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


# 4.pybind11 cuda

In [20]:
import nms_cuda 
import numpy as np

bbox=np.load("bbox.npy")
n_bbox = bbox.shape[0]

keep_out=np.zeros(bbox.shape[0],dtype=np.int32)
%timeit n=nms_cuda.nms_cuda(keep_out,bbox)
# print(keep_out)
# print(n)

26.1 ms ± 551 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
